In [1]:
import requests
import pandas as pd
import io
import time
from datetime import datetime
import locale
from tqdm import tqdm
import concurrent.futures
import os

In [2]:
url = f'http://api.steampowered.com/ISteamApps/GetAppList/v2/' #leemos la lista de juegos completos
response = requests.get(url).content
df = pd.read_json(io.BytesIO(response))
df

,applist
apps,"[{'appid': 5, 'name': 'Dedicated Server'}, {'a..."


In [4]:
df = pd.json_normalize(df['applist']) #normalizamos la primera columna
json_list = df.iloc[0].tolist() #ejecutamos .tolist() a la primera fila
json_list

[{'appid': 5, 'name': 'Dedicated Server'},
 {'appid': 7, 'name': 'Steam Client'},
 {'appid': 8, 'name': 'winui2'},
 {'appid': 10, 'name': 'Counter-Strike'},
 {'appid': 20, 'name': 'Team Fortress Classic'},
 {'appid': 30, 'name': 'Day of Defeat'},
 {'appid': 40, 'name': 'Deathmatch Classic'},
 {'appid': 50, 'name': 'Half-Life: Opposing Force'},
 {'appid': 60, 'name': 'Ricochet'},
 {'appid': 70, 'name': 'Half-Life'},
 {'appid': 80, 'name': 'Counter-Strike: Condition Zero'},
 {'appid': 90, 'name': 'Half-Life Dedicated Server'},
 {'appid': 92, 'name': 'Codename Gordon'},
 {'appid': 100, 'name': 'Counter-Strike: Condition Zero Deleted Scenes'},
 {'appid': 130, 'name': 'Half-Life: Blue Shift'},
 {'appid': 150, 'name': 'Counter-Strike Steamworks Beta'},
 {'appid': 205, 'name': 'Source Dedicated Server'},
 {'appid': 210, 'name': 'Source Dedicated Server'},
 {'appid': 211, 'name': 'Source SDK'},
 {'appid': 215, 'name': 'Source SDK Base 2006'},
 {'appid': 218, 'name': 'Source SDK Base 2007'},
 {

In [5]:
result_df = pd.DataFrame(json_list)[['appid', 'name']] #volvemos lo previo a dataframe
result_df

,appid,name
0,5,Dedicated Server
1,7,Steam Client
2,8,winui2
3,10,Counter-Strike
4,20,Team Fortress Classic
...,...,...
273251,4115600,恐怖地牢 Demo
273252,4115850,Sweet OL
273253,4116120,"Creature Clicker - Capture, Train, Ascend! Pla..."
273254,4117180,Monkey Do Playtest


In [6]:
cincok = pd.read_csv('archivo.txt', sep='\t', engine='python', header=None, encoding='utf-8-sig') #este codigo fue generado gracias a chat gpt, busca el poder volver el .txt a dataframe
cincok.columns = ['Ranking', 'vacio', 'name', 'jugadores_actuales', '24H-jugadores', 'Historico', 'nada2'] #acomodamos las columnas, hay 2 columnas cuyos datos son irrevelevantes o estan vacias, estas son las con nombres vacios y nada2
cincok['Ranking'] = cincok['Ranking'].astype(str).str.extract(r'(\d+)').astype(int) # Limpiar la columna Ranking (eliminar el punto si está como "1.")
cincok.head()


,Ranking,vacio,name,jugadores_actuales,24H-jugadores,Historico,nada2
0,1,NaN,Counter-Strike 2,"570,155","1,351,838","1,862,531",+
1,2,NaN,Dota 2,"355,440","742,248","1,295,114",+
2,3,NaN,Battlefield™ 6,"262,954","358,857","747,440",+
3,4,NaN,Banana,"95,331","113,836","917,272",+
4,5,NaN,PUBG: BATTLEGROUNDS,"76,317","627,837","3,257,248",+


In [7]:
cincok.drop(columns=['vacio', 'nada2', 'Ranking', 'jugadores_actuales', '24H-jugadores'], inplace=True) #eliminamos las columnas que no vamos a usar
cincok.head()

,name,Historico
0,Counter-Strike 2,"1,862,531"
1,Dota 2,"1,295,114"
2,Battlefield™ 6,"747,440"
3,Banana,"917,272"
4,PUBG: BATTLEGROUNDS,"3,257,248"


In [8]:
cincok.tail()

,name,Historico
4995,MiSide Demo,"1,354"
4996,Snezhinka,"5,455"
4997,Streets of Rogue 2 Demo,"1,488"
4998,Garten of Banban 7,"1,859"
4999,Realm of Ink Demo,"1,479"


In [9]:
df_merged = pd.merge(cincok, result_df, on='name', how='inner') #combinamos los 2 dataframes en uno que tenga solo aquellas filas con datos en comun en la columna name
df_merged = df_merged.drop_duplicates(subset='name', keep='first')  #eliminamos los juegos que comparten el mismo nombre pero diferente id (nos quedaremos con el primero de estos)
df_merged.reset_index(drop=True, inplace=True) #reiniciamos el index solo por comodidad
df_merged

,name,Historico,appid
0,Counter-Strike 2,"1,862,531",730
1,Dota 2,"1,295,114",570
2,Battlefield™ 6,"747,440",2807960
3,Banana,"917,272",2923300
4,PUBG: BATTLEGROUNDS,"3,257,248",578080
...,...,...,...
4834,MiSide Demo,"1,354",2527520
4835,Snezhinka,"5,455",2608350
4836,Streets of Rogue 2 Demo,"1,488",2670760
4837,Garten of Banban 7,"1,859",2693060


In [23]:
df_merged.to_csv('review.csv', index=False)

In [24]:
df_merged = pd.read_csv('review.csv')
output_file = 'reviewfilled.csv'

In [26]:
# Verificar si ya existe un archivo con datos previos
if os.path.exists(output_file):
    df_existente = pd.read_csv(output_file)
    procesados = set(df_existente['appid'])
    print(f"Reanudando desde archivo parcial con {len(procesados)} registros ya guardados.")
else:
    # Crear un DataFrame vacío con columnas necesarias
    df_existente = pd.DataFrame(columns=['appid', 'review_score', 'review_score_desc', 'total_reviews'])
    procesados = set()

# Filtrar los appid que aún no se han procesado
faltantes = df_merged[~df_merged['appid'].isin(procesados)].copy()
print(f"Faltan por procesar: {len(faltantes)}")

# Función que consulta la API de Steam
def fetch_review(appid):
    url = f'https://store.steampowered.com/appreviews/{appid}?json=1'
    try:
        response = requests.get(url, timeout=5)
        data = response.json()
        if data.get('success') == 1:
            return {
                'appid': appid,
                'review_score': data['query_summary'].get('review_score'),
                'review_score_desc': data['query_summary'].get('review_score_desc'),
                'total_reviews': data['query_summary'].get('total_reviews')
            }
    except Exception as e:
        print(f"Error con appid {appid}: {e}")
    return {'appid': appid, 'review_score': None, 'review_score_desc': None, 'total_reviews': None}

# Variables de control
BATCH_SIZE = 100
results = []

# Descargar los datos en paralelo con barra de progreso
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    for i, result in enumerate(tqdm(executor.map(fetch_review, faltantes['appid']), total=len(faltantes))):
        results.append(result)

        # Guardado parcial cada BATCH_SIZE resultados
        if (i + 1) % BATCH_SIZE == 0 or (i + 1) == len(faltantes):
            df_temp = pd.DataFrame(results)
            df_existente = pd.concat([df_existente, df_temp], ignore_index=True)
            df_existente.to_csv(output_file, index=False)
            results = []  # Vaciar buffer
            print(f"Guardado parcial: {len(df_existente)} filas guardadas hasta ahora.")

print("✅ Proceso completado.")

Reanudando desde archivo parcial con 2100 registros ya guardados.
Faltan por procesar: 2739


  4%|▍         | 105/2739 [00:11<03:29, 12.60it/s]

Guardado parcial: 2200 filas guardadas hasta ahora.


  7%|▋         | 201/2739 [00:21<03:58, 10.66it/s]

Guardado parcial: 2300 filas guardadas hasta ahora.


...

Guardado parcial: 4700 filas guardadas hasta ahora.


 99%|█████████▊| 2703/2739 [04:46<00:03, 10.21it/s]

Guardado parcial: 4800 filas guardadas hasta ahora.


100%|██████████| 2739/2739 [04:50<00:00,  9.43it/s]

Guardado parcial: 4839 filas guardadas hasta ahora.
✅ Proceso completado.


In [ ]:
reviews = pd.read_csv('reviewfilled.csv')
reviews.head()

,appid,review_score,review_score_desc,total_reviews
0,730,8.0,Very Positive,1367090.0
1,570,8.0,Very Positive,6098.0
2,2807960,8.0,Very Positive,58813.0
3,2923300,0.0,No user reviews,0.0
4,578080,5.0,Mixed,277935.0


In [ ]:
df_merged = pd.merge(df_merged, reviews, on='appid', how='inner')
df_merged #por alguna razon merge de esta forma, por lo que vamos a borrar ciertas columnas

,name,Historico,appid,review_score_x,review_score_desc_x,total_reviews_x,review_score_y,review_score_desc_y,total_reviews_y
0,Counter-Strike 2,"1,862,531",730,8.0,Very Positive,1367090.0,8.0,Very Positive,1367090.0
1,Dota 2,"1,295,114",570,8.0,Very Positive,6098.0,8.0,Very Positive,6098.0
2,Battlefield™ 6,"747,440",2807960,8.0,Very Positive,58813.0,8.0,Very Positive,58813.0
3,Banana,"917,272",2923300,0.0,No user reviews,0.0,0.0,No user reviews,0.0
4,PUBG: BATTLEGROUNDS,"3,257,248",578080,5.0,Mixed,277935.0,5.0,Mixed,277935.0
...,...,...,...,...,...,...,...,...,...
4834,MiSide Demo,"1,354",2527520,0.0,No user reviews,0.0,0.0,No user reviews,0.0
4835,Snezhinka,"5,455",2608350,9.0,Overwhelmingly Positive,511.0,9.0,Overwhelmingly Positive,511.0
4836,Streets of Rogue 2 Demo,"1,488",2670760,0.0,No user reviews,0.0,0.0,No user reviews,0.0
4837,Garten of Banban 7,"1,859",2693060,8.0,Very Positive,852.0,8.0,Very Positive,852.0


In [33]:
df_merged.columns

Index(['name', 'Historico', 'appid', 'review_score_x', 'review_score_desc_x',
       'total_reviews_x', 'review_score_y', 'review_score_desc_y',
       'total_reviews_y'],
      dtype='object')

In [ ]:
df_merged.drop(columns=['review_score_y', 'review_score_desc_y','total_reviews_y'], inplace=True)

In [35]:
df_merged.head()

,name,Historico,appid,review_score_x,review_score_desc_x,total_reviews_x
0,Counter-Strike 2,"1,862,531",730,8.0,Very Positive,1367090.0
1,Dota 2,"1,295,114",570,8.0,Very Positive,6098.0
2,Battlefield™ 6,"747,440",2807960,8.0,Very Positive,58813.0
3,Banana,"917,272",2923300,0.0,No user reviews,0.0
4,PUBG: BATTLEGROUNDS,"3,257,248",578080,5.0,Mixed,277935.0


In [36]:
df_merged.to_csv('review.csv', index=False)

In [2]:
df_merged = pd.read_csv('review.csv')

In [4]:
df_merged.head()

,name,Historico,appid,review_score_x,review_score_desc_x,total_reviews_x
0,Counter-Strike 2,"1,862,531",730,8.0,Very Positive,1367090.0
1,Dota 2,"1,295,114",570,8.0,Very Positive,6098.0
2,Battlefield™ 6,"747,440",2807960,8.0,Very Positive,58813.0
3,Banana,"917,272",2923300,0.0,No user reviews,0.0
4,PUBG: BATTLEGROUNDS,"3,257,248",578080,5.0,Mixed,277935.0


In [13]:
output_file = 'appdetails.csv'

In [15]:
if os.path.exists(output_file):
    df_existente = pd.read_csv(output_file)
    procesados = set(df_existente['appid'])
else:
    df_existente = pd.DataFrame(columns=['appid', 'name', 'type', 'required_age', 'is_free', 'detailed_description', 'price', 'developers'])
    procesados = set()

faltantes = df_merged[~df_merged['appid'].isin(procesados)].copy()

def fetch_appdetails(appid):
    url = f'https://store.steampowered.com/api/appdetails?appids={appid}'
    try:
        response = requests.get(url, timeout=5)
        data = response.json()
        app_data = data.get(str(appid), {})
        if app_data.get('success'):
            info = app_data.get('data', {})
            return {
                'appid': appid,
                'name': info.get('name'),
                'type': info.get('type'),
                'required_age': info.get('required_age'),
                'is_free': info.get('is_free'),
                'developers': info.get('developers', [None])[0],
                'price': info.get('price_overview', {}).get('final_formatted') if info.get('price_overview') else 'Free',
                'detailed_description': info.get('detailed_description', '')[:200]
            }
    except Exception as e:
        print(f"Error con appid {appid}: {e}")
    time.sleep(1)
    return {'appid': appid, 'name': None, 'type': None, 'required_age': None, 'is_free': None, 'detailed_description': None}

BATCH_SIZE = 100
results = []

with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
    for i, result in enumerate(tqdm(executor.map(fetch_appdetails, faltantes['appid']), total=len(faltantes))):
        results.append(result)
        if (i + 1) % BATCH_SIZE == 0 or (i + 1) == len(faltantes):
            df_temp = pd.DataFrame(results)
            df_existente = pd.concat([df_existente, df_temp], ignore_index=True)
            df_existente.to_csv(output_file, index=False)
            results = []
            print(f"Guardado parcial: {len(df_existente)} filas guardadas hasta ahora.")

print("✅ Proceso completado.")

 12%|█▏        | 62/539 [00:45<06:06,  1.30it/s]

Error con appid 253940: 'NoneType' object has no attribute 'get'


 12%|█▏        | 63/539 [00:46<08:05,  1.02s/it]

Error con appid 281640: 'NoneType' object has no attribute 'get'


 12%|█▏        | 64/539 [00:48<09:38,  1.22s/it]

...

Error con appid 218410: 'NoneType' object has no attribute 'get'


 93%|█████████▎| 500/539 [08:48<00:29,  1.34it/s]

Guardado parcial: 4800 filas guardadas hasta ahora.


100%|██████████| 539/539 [09:17<00:00,  1.03s/it]

Guardado parcial: 4839 filas guardadas hasta ahora.
✅ Proceso completado.


In [16]:
juegos = pd.read_csv('appdetails.csv')
juegos

,appid,name,type,required_age,is_free,detailed_description,price,developers
0,730,Counter-Strike 2,game,0,True,"For over two decades, Counter-Strike has offer...",Free,Valve
1,570,Dota 2,game,0,True,<strong>The most-played game on Steam.</strong...,Free,Valve
2,2807960,Battlefield™ 6,game,0,False,<h1>Battlefield™ 6 Phantom Edition</h1><p><spa...,$69.99 USD,Battlefield Studios
3,2923300,Banana,game,0,True,"<p class=""bb_paragraph"" >Banana is a clicker G...",Free,Sky
4,578080,PUBG: BATTLEGROUNDS,game,0,True,"<p class=""bb_paragraph"" ><span class=""bb_img_c...",Free,PUBG Corporation
...,...,...,...,...,...,...,...,...
4834,2527520,MiSide Demo,demo,0,True,MiSide is an adventure game with horror elemen...,Free,AIHASTO
4835,2608350,Snezhinka:Sentinel Girls2,game,0,False,"[Game Overview]<br>Play as Snezhinka, a fixed-...",CLP$ 4.700,hinyari9
4836,2670760,Streets of Rogue 2 Demo,demo,0,True,<br><br>Streets of Rogue 2 invites you to a li...,Free,Matt Dabrowski
4837,2693060,Garten of Banban 7,game,0,False,<h1>WISHLIST NOW!</h1><p></p><br><h1>About the...,CLP$ 5.750,Euphoric Brothers


In [17]:
df_completo = pd.merge(df_merged, juegos, on='appid', how='inner')
df_completo

,name_x,Historico,appid,review_score_x,review_score_desc_x,total_reviews_x,name_y,type,required_age,is_free,detailed_description,price,developers
0,Counter-Strike 2,"1,862,531",730,8.0,Very Positive,1367090.0,Counter-Strike 2,game,0,True,"For over two decades, Counter-Strike has offer...",Free,Valve
1,Dota 2,"1,295,114",570,8.0,Very Positive,6098.0,Dota 2,game,0,True,<strong>The most-played game on Steam.</strong...,Free,Valve
2,Battlefield™ 6,"747,440",2807960,8.0,Very Positive,58813.0,Battlefield™ 6,game,0,False,<h1>Battlefield™ 6 Phantom Edition</h1><p><spa...,$69.99 USD,Battlefield Studios
3,Banana,"917,272",2923300,0.0,No user reviews,0.0,Banana,game,0,True,"<p class=""bb_paragraph"" >Banana is a clicker G...",Free,Sky
4,PUBG: BATTLEGROUNDS,"3,257,248",578080,5.0,Mixed,277935.0,PUBG: BATTLEGROUNDS,game,0,True,"<p class=""bb_paragraph"" ><span class=""bb_img_c...",Free,PUBG Corporation
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4834,MiSide Demo,"1,354",2527520,0.0,No user reviews,0.0,MiSide Demo,demo,0,True,MiSide is an adventure game with horror elemen...,Free,AIHASTO
4835,Snezhinka,"5,455",2608350,9.0,Overwhelmingly Positive,511.0,Snezhinka:Sentinel Girls2,game,0,False,"[Game Overview]<br>Play as Snezhinka, a fixed-...",CLP$ 4.700,hinyari9
4836,Streets of Rogue 2 Demo,"1,488",2670760,0.0,No user reviews,0.0,Streets of Rogue 2 Demo,demo,0,True,<br><br>Streets of Rogue 2 invites you to a li...,Free,Matt Dabrowski
4837,Garten of Banban 7,"1,859",2693060,8.0,Very Positive,852.0,Garten of Banban 7,game,0,False,<h1>WISHLIST NOW!</h1><p></p><br><h1>About the...,CLP$ 5.750,Euphoric Brothers


In [18]:
df_completo.drop(columns=['name_y'], inplace=True)
df_completo.head()

,name_x,Historico,appid,review_score_x,review_score_desc_x,total_reviews_x,type,required_age,is_free,detailed_description,price,developers
0,Counter-Strike 2,"1,862,531",730,8.0,Very Positive,1367090.0,game,0,True,"For over two decades, Counter-Strike has offer...",Free,Valve
1,Dota 2,"1,295,114",570,8.0,Very Positive,6098.0,game,0,True,<strong>The most-played game on Steam.</strong...,Free,Valve
2,Battlefield™ 6,"747,440",2807960,8.0,Very Positive,58813.0,game,0,False,<h1>Battlefield™ 6 Phantom Edition</h1><p><spa...,$69.99 USD,Battlefield Studios
3,Banana,"917,272",2923300,0.0,No user reviews,0.0,game,0,True,"<p class=""bb_paragraph"" >Banana is a clicker G...",Free,Sky
4,PUBG: BATTLEGROUNDS,"3,257,248",578080,5.0,Mixed,277935.0,game,0,True,"<p class=""bb_paragraph"" ><span class=""bb_img_c...",Free,PUBG Corporation


In [19]:
df_completo.to_csv('juegos_completos.csv', index=False)